# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,poronaysk,49.2167,143.1167,-18.71,75,5,0.46,RU,1675043055
1,1,pucallpa,-8.3791,-74.5539,26.02,83,75,2.06,PE,1675043056
2,2,kahului,20.8947,-156.4700,23.80,79,40,4.12,US,1675043056
3,3,hilo,19.7297,-155.0900,22.38,88,20,0.45,US,1675042860
4,4,saint-pierre,-21.3393,55.4781,25.82,73,0,1.54,RE,1675043057


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map_plot_1

ImportError: Geographic projection support requires GeoViews and cartopy.

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values

#A max temperature lower than 27 degrees but higher than 21
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

#Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop results with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,saint-pierre,-21.3393,55.4781,25.82,73,0,1.54,RE,1675043057
73,73,new norfolk,-42.7826,147.0587,24.84,31,0,0.88,AU,1675043101
84,84,pochutla,15.7432,-96.4661,26.01,73,0,1.87,MX,1675043108
178,178,santarem,-2.4431,-54.7083,26.07,94,0,1.03,BR,1675043043
220,220,arica,-18.4750,-70.3042,23.13,68,0,1.03,CL,1675043137
292,292,mercedes,-34.6515,-59.4307,22.58,68,0,4.09,AR,1675043223
330,330,cidreira,-30.1811,-50.2056,22.49,89,0,0.19,BR,1675043248
449,449,lazaro cardenas,17.9583,-102.2000,24.25,80,0,1.39,MX,1675043310
457,457,vila velha,-20.3297,-40.2925,24.97,88,0,2.57,BR,1675043315
512,512,dzaoudzi,-12.7887,45.2699,25.94,89,0,1.54,YT,1675043346


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_df = city_data_df[{"City": city,
                        "Country": country,
                        "Lat": lat,
                        "Lng": lng,
                        "Humidity": humidity}]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

/var/folders/gz/yqwhjwdd6c75tl8kh7k5sr700000gn/T/ipykernel_82975/3677612766.py:9: FutureWarning: Passing a dict as an indexer is deprecated and will raise in a future version. Use a list instead.
  hotel_df = city_data_df[{"City": city,
/var/folders/gz/yqwhjwdd6c75tl8kh7k5sr700000gn/T/ipykernel_82975/3677612766.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
4,saint-pierre,RE,-21.3393,55.4781,73,
73,new norfolk,AU,-42.7826,147.0587,31,
84,pochutla,MX,15.7432,-96.4661,73,
178,santarem,BR,-2.4431,-54.7083,94,
220,arica,CL,-18.4750,-70.3042,68,
292,mercedes,AR,-34.6515,-59.4307,68,
330,cidreira,BR,-30.1811,-50.2056,89,
449,lazaro cardenas,MX,17.9583,-102.2000,80,
457,vila velha,BR,-20.3297,-40.2925,88,
512,dzaoudzi,YT,-12.7887,45.2699,89,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
new norfolk - nearest hotel: Woodbridge on the Derwent
pochutla - nearest hotel: Posada San Jose
santarem - nearest hotel: Prático Hotel
arica - nearest hotel: Hotel Andalucía 
mercedes - nearest hotel: Gran Hotel Mercedes
cidreira - nearest hotel: Hotel Castelo
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
vila velha - nearest hotel: Hotel Prainha
dzaoudzi - nearest hotel: 2.0 hôtel
belmonte - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
4,saint-pierre,RE,-21.3393,55.4781,73,Tropic Hotel
73,new norfolk,AU,-42.7826,147.0587,31,Woodbridge on the Derwent
84,pochutla,MX,15.7432,-96.4661,73,Posada San Jose
178,santarem,BR,-2.4431,-54.7083,94,Prático Hotel
220,arica,CL,-18.4750,-70.3042,68,Hotel Andalucía
292,mercedes,AR,-34.6515,-59.4307,68,Gran Hotel Mercedes
330,cidreira,BR,-30.1811,-50.2056,89,Hotel Castelo
449,lazaro cardenas,MX,17.9583,-102.2000,80,Hotel Sol del Pacífico
457,vila velha,BR,-20.3297,-40.2925,88,Hotel Prainha
512,dzaoudzi,YT,-12.7887,45.2699,89,2.0 hôtel


In [16]:
name_address

{'type': 'FeatureCollection', 'features': []}

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

ImportError: Geographic projection support requires GeoViews and cartopy.